# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the 2010 Formula 1 World Championship driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel. 
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [5]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 championship was won by Fernando Alonso, driving for the Renault F1 Team.


In [6]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [7]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [8]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [9]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


In [14]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are a tour guide.'}
]
print(return_OAIResponse("I want to visit Peru.", context_user))

That's great! Peru is a beautiful country with a rich history and diverse landscapes. Are there any specific places in Peru that you are interested in visiting, or any activities you are looking forward to doing there?


In [19]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are a tour guide.

     From: Lima.
     Activities: Visit Plaza Mayor, Larco Museum, Miraflores Boardwalk.
     Best for: Culture, History, Gastronomy.

     From: Cusco.
     Activities: Visit Machu Picchu, Sacred Valley, Sacsayhuamán.
     Best for: History, Nature, Adventure.
     
     From: Arequipa.
     Activities: Visit Colca Canyon, Santa Catalina Monastery, Arequipa Cathedral.
     Best for: Nature, Culture, History.
     
     From: Iquitos.
     Activities: Visit the Amazon rainforest, Belen Market, Pilpintuwasi Butterfly Farm.
     Best for: Nature, Adventure, Wildlife.
     """}

]
print(return_OAIResponse("I want to visit Peru.", context_user))

Great choice! Peru is a beautiful country with a rich history and diverse culture. Depending on your interests, there are different cities you can visit for a unique experience. Here are some suggestions based on the city you choose as your starting point:

1. If you start from Lima:
   - Visit Plaza Mayor for its historical significance and beautiful architecture.
   - Explore the Larco Museum to learn about Peru's pre-Columbian history.
   - Take a walk along the Miraflores Boardwalk for stunning views of the Pacific Ocean.

2. If you start from Cusco:
   - Don't miss a visit to the iconic Machu Picchu, a UNESCO World Heritage site.
   - Explore the Sacred Valley to see Inca ruins and traditional villages.
   - Visit the impressive archaeological site of Sacsayhuamán for its massive stone structures.

3. If you start from Arequipa:
   - Explore the breathtaking Colca Canyon, known for its beautiful landscapes and Andean condors.
   - Visit the Santa Catalina Monastery to learn about 

In [20]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are a tour guide.

     From: Lima.
     Activities: Visit Plaza Mayor, Larco Museum, Miraflores Boardwalk.
     Best for: Culture, History, Gastronomy.

     From: Cusco.
     Activities: Visit Machu Picchu, Sacred Valley, Sacsayhuamán.
     Best for: History, Nature, Adventure.
     
     From: Arequipa.
     Activities: Visit Colca Canyon, Santa Catalina Monastery, Arequipa Cathedral.
     Best for: Nature, Culture, History.
     
     From: Iquitos.
     Activities: Visit the Amazon rainforest, Belen Market, Pilpintuwasi Butterfly Farm.
     Best for: Nature, Adventure, Wildlife.
     """}

]
print(return_OAIResponse("I want to visit Peru.", context_user))

Peru is a country rich in culture, history, and natural beauty. Depending on your interests, there are several regions you can visit that offer unique experiences. Here are some options based on where you might want to start your journey:

- If you are interested in exploring the bustling city life and historical sites, you can start your trip in Lima. Here, you can visit the Plaza Mayor, the Larco Museum, and take a leisurely stroll along the Miraflores Boardwalk to experience the gastronomy of the city.

- For those who are fascinated by ancient civilizations and breathtaking landscapes, starting in Cusco might be the perfect choice. You can visit iconic sites such as Machu Picchu, the Sacred Valley, and the impressive ruins of Sacsayhuamán while immersing yourself in the history, nature, and adventure of the region.

- If you prefer a blend of nature, culture, and history, beginning your journey in Arequipa could be ideal. Explore the stunning Colca Canyon, wander through the colorf

In [21]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are a tour guide.

     From: Lima.
     Activities: Visit Plaza Mayor, Larco Museum, Miraflores Boardwalk.
     Best for: Culture, History, Gastronomy.

     From: Cusco.
     Activities: Visit Machu Picchu, Sacred Valley, Sacsayhuamán.
     Best for: History, Nature, Adventure.
     
     From: Arequipa.
     Activities: Visit Colca Canyon, Santa Catalina Monastery, Arequipa Cathedral.
     Best for: Nature, Culture, History.
     
     From: Iquitos.
     Activities: Visit the Amazon rainforest, Belen Market, Pilpintuwasi Butterfly Farm.
     Best for: Nature, Adventure, Wildlife.
     """}

]
print(return_OAIResponse("I want to visit Peru.", context_user))

That's great! Peru is a wonderful destination with so much to offer. Depending on your interests, here are some recommendations for tours from different cities in Peru:

- **From Lima**: If you're into culture, history, and gastronomy, you can visit Plaza Mayor, Larco Museum, and walk along the Miraflores Boardwalk. Lima is known for its delicious cuisine, so be sure to try some local dishes like ceviche and lomo saltado.

- **From Cusco**: For history, nature, and adventure seekers, Cusco is the gateway to Machu Picchu, the Sacred Valley, and the impressive Sacsayhuamán ruins. Trekking to Machu Picchu is an unforgettable experience and a must-do for many travelers.

- **From Arequipa**: Nature lovers will enjoy exploring Colca Canyon, visiting the Santa Catalina Monastery, and admiring the architecture of Arequipa Cathedral. The city is surrounded by stunning landscapes and has a rich colonial history.

- **From Iquitos**: If you're looking for a unique nature experience, Iquitos is t

In [ ]:
context_user = [
    {'role':'system', 'content':'You are a tour guide.\n\n'},
    {'role':'user', 'content':'I want to visit Peru.'},
    {'role':'assistant', 'content':'From: Lima.\nActivities: Visit Plaza Mayor, Larco Museum, Miraflores Boardwalk.\nBest for: Culture, History, Gastronomy. \nTime: 3 days.\nAirport: Jorge Chavez International Airport (LIM).'},
    {'role':'user', 'content':'I want to visit Philippines.'},
    {'role':'assistant', 'content':'From: Manila.\nActivities: Visit Intramuros, Rizal Park, Manila Ocean Park.\nBest for: Culture, History, Nature. \nTime: 5 days.\nAirport: Ninoy Aquino International Airport (MNL).'},
    {'role':'user', 'content':'I want to visit Denmark.'},
    {'role':'assistant', 'content':'From: Copenhagen.\nActivities: Visit Tivoli Gardens, Nyhavn, The Little Mermaid.\nBest for: Culture, History, Architecture. \nTime: 4 days.\nAirport: Copenhagen Airport (CPH).'},
]

print(return_OAIResponse("I want to visit Peru.", context_user))

From: Lima.
Activities: Visit Machu Picchu, Sacred Valley, Lake Titicaca.
Best for: History, Nature, Adventure.
Time: 7-10 days.
Airport: Jorge Chavez International Airport (LIM).


In [28]:
context_user = [
    {'role':'system', 'content':"""You are a tour guide.\n\n'Reply with information like in this example'From: Lima.\nActivities: Visit Plaza Mayor, Larco Museum, Miraflores Boardwalk.\nBest for: Culture, History, Gastronomy. \nTime: 3 days.\nAirport: Jorge Chavez International Airport (LIM).'
     """}
]

print(return_OAIResponse("I want to visit Peru.", context_user))

From: Lima.
Activities: Visit Plaza Mayor, Larco Museum, Miraflores Boardwalk.
Best for: Culture, History, Gastronomy. 
Time: 3 days.
Airport: Jorge Chavez International Airport (LIM).


In [29]:
context_user = [
    {'role':'system', 'content':'You are a tour guide.\n\n'},
    {'role':'user', 'content':'I want to visit Peru.'},
    {'role':'assistant', 'content':'From: Lima.\nActivities: Visit Plaza Mayor, Larco Museum, Miraflores Boardwalk.\nBest for: Culture, History, Gastronomy. \nTime: 3 days.\nAirport: Jorge Chavez International Airport (LIM).'}
]

print(return_OAIResponse("I want to visit Peru.", context_user))

Peru is a fantastic destination that offers a rich blend of history, culture, and natural beauty. From the majestic ruins of Machu Picchu to the vibrant streets of Lima and the mystical Nazca Lines, there is something for every traveler to experience and enjoy in Peru. Let me know if you need any assistance in planning your trip!


In [25]:
#CoT
context_user = [
    {'role':'system', 'content':""" You are a tour guide.'
    1) Know the main attractions of the place.
    2) Know the best time to visit.
    3) Know the best way to get there.
    4) Know the best way to get around.
    5) Know the best places to eat.
    6) Know the best places to stay.
    7) Provide a list of what to be aware of and bring when visiting the place.
    """
    }
]
print(return_OAIResponse("I want to visit Peru.", context_user))

**Destination: Peru**

**Main attractions:**
1) Machu Picchu
2) Sacred Valley
3) Amazon Rainforest
4) Lake Titicaca
5) Colca Canyon
6) Nazca Lines
7) Plaza de Armas in Cusco
8) Sacsayhuamán
9) Lima's Historic Center

**Best time to visit:**
The best time to visit Peru is during the dry season which runs from May to September. This is the peak tourist season with clear skies and comfortable temperatures. However, visiting during the shoulder seasons of April and October can also be a good option to avoid large crowds.

**Best way to get there:**
The main international airport in Peru is Jorge Chávez International Airport in Lima. Many major airlines have flights to Lima from various international destinations.

**Best way to get around:**
Within Peru, domestic flights are a quick way to cover long distances. Buses are also a popular and affordable option, especially for shorter distances. In cities like Lima and Cusco, taxis and ridesharing services are readily available.

**Best places

In [26]:
#CoT
context_user = [
    {'role':'system', 'content':""" You are a tour guide.'
    1) Know the main attractions of the place.
    2) Know the best time to visit.
    3) Know the best way to get there.
    4) Know the best way to get around.
    5) Know the best places to eat.
    6) Know the best places to stay.
    7) Provide a list of what to be aware of and bring when visiting the place.
    """
    }
]
print(return_OAIResponse("I want to visit Peru.", context_user))

Peru is an amazing destination with a rich history, diverse culture, and stunning landscapes. Here is some information to help you plan your trip to Peru:

Main attractions: 
1) Machu Picchu - The ancient Incan city located in the Andes Mountains is a must-visit.
2) The Sacred Valley - Home to impressive Incan ruins and traditional villages.
3) Lake Titicaca - The highest navigable lake in the world with stunning scenery and local culture.
4) Nazca Lines - Mysterious geoglyphs in the desert that can be best seen from the air.
5) Amazon Rainforest - Explore the incredible biodiversity of the Amazon.

Best time to visit:
The best time to visit Peru is during the dry season, which runs from May to September. This period offers the most pleasant weather for exploring the various attractions in Peru.

Best way to get there:
You can fly into the main international airports in Lima and Cusco. From there, you can easily travel to other parts of the country by bus, train, or domestic flights.



In [27]:
#CoT
context_user = [
    {'role':'system', 'content':""" You are a tour guide.'
    1) Know the main attractions of the place.
    2) Know the best time to visit.
    3) Know the best way to get there.
    4) Know the best way to get around.
    5) Know the best places to eat.
    6) Know the best places to stay.
    7) Provide a list of what to be aware of and bring when visiting the place.
    """
    }
]
print(return_OAIResponse("I want to visit Peru.", context_user))

Peru is an amazing country with diverse landscapes, rich history, and vibrant culture. Here's a guide to help you make the most of your trip to Peru:

Main Attractions:
1) Machu Picchu - An ancient Inca citadel located in the Andes Mountains, it is one of the most famous archaeological sites in the world.
2) The Sacred Valley - A beautiful region in the Andes known for its stunning landscapes, Inca ruins, and traditional indigenous communities.
3) The Amazon Rainforest - Experience the incredible biodiversity of the Amazon with activities like wildlife spotting, jungle hikes, and river cruises.
4) Lake Titicaca - The highest navigable lake in the world, with unique floating islands and indigenous communities.
5) Cusco - The former capital of the Inca Empire, Cusco is a UNESCO World Heritage Site with colonial architecture and Inca ruins.

Best Time to Visit:
The best time to visit Peru is during the dry season, which runs from May to September. This is also the peak tourist season, so 

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
Know what you information and format that you do not want to miss and be explicit about it in the prompt.